In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
import scipy as sp
import os
import xgboost as xgb
%matplotlib inline

In [81]:
pd.set_option('display.max_columns', None)

In [201]:
train = pd.read_csv("/Users/andrewrubino/Desktop/train.csv", parse_dates = ['timestamp']).drop('id', axis=1)
print train.shape
test = pd.read_csv("/Users/andrewrubino/Desktop/test.csv", parse_dates = ['timestamp']).drop('id', axis=1)
print test.shape
macro = pd.read_csv("/Users/andrewrubino/Desktop/macro.csv", parse_dates = ['timestamp'])
print macro.shape

(30471, 291)
(7662, 290)
(2484, 100)


In [202]:
ftr_train = train.loc[:, ['price_doc',
'timestamp',
'full_sq',
'life_sq',
'floor',
'max_floor',
'material',
'build_year',
'num_room',
'kitch_sq',
'state',
'product_type',
'sub_area',
'indust_part',
'school_education_centers_raion',
'sport_objects_raion',
'culture_objects_top_25_raion',
'oil_chemistry_raion',
'metro_min_avto',
'green_zone_km',
'industrial_km',
'kremlin_km',
'radiation_km',
'ts_km',
'fitness_km',
'stadium_km',
'additional_education_km',
'cafe_count_1500_price_500',
'cafe_count_1500_price_high',
'cafe_count_2000_price_2500',
'trc_sqm_5000',
'cafe_count_5000',
'cafe_count_5000_price_high']]


ftr_test = test.loc[:, ['timestamp',
'full_sq',
'life_sq',
'floor',
'max_floor',
'material',
'build_year',
'num_room',
'kitch_sq',
'state',
'product_type',
'sub_area',
'indust_part',
'school_education_centers_raion',
'sport_objects_raion',
'culture_objects_top_25_raion',
'oil_chemistry_raion',
'metro_min_avto',
'green_zone_km',
'industrial_km',
'kremlin_km',
'radiation_km',
'ts_km',
'fitness_km',
'stadium_km',
'additional_education_km',
'cafe_count_1500_price_500',
'cafe_count_1500_price_high',
'cafe_count_2000_price_2500',
'trc_sqm_5000',
'cafe_count_5000',
'cafe_count_5000_price_high']]

ftr_macro = macro.loc[:,['timestamp',
'oil_urals',
'gdp_quart',
'cpi',
'ppi',
'usdrub',
'eurrub',
'gdp_annual',
'rts',
'micex',
'micex_cbi_tr',
'deposits_rate',
'mortgage_rate',
'income_per_cap',
'salary',
'labor_force',
'unemployment',
'employment']]

id_test = test['id']

KeyError: 'id'

In [140]:
train = pd.merge(ftr_train, ftr_macro, how = 'left', on = 'timestamp')
test = pd.merge(ftr_test, ftr_macro, how = 'left', on = 'timestamp')

In [141]:
from sklearn import model_selection, preprocessing
from sklearn.preprocessing import LabelEncoder

def encode_object_features(train_df, test_df):
    '''(DataFrame, DataFrame) -> DataFrame, DataFrame
    
    Will encode each non-numerical column.
    '''
    for f in train_df.columns:
        if train_df[f].dtype=='object':
            print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values.astype('str')) + list(test_df[f].values.astype('str')))
            train_df[f] = lbl.transform(list(train_df[f].values.astype('str')))
            test_df[f] = lbl.transform(list(test_df[f].values.astype('str')))
    
    return train_df, test_df

In [142]:
train, test = encode_object_features(train, test)

product_type
sub_area
oil_chemistry_raion


In [143]:
# Impute missing values in training set

from scipy.stats import mode

print int(mode(train['life_sq']).mode[0])
print mode(train['floor']).mode[0]
print mode(train['max_floor']).mode[0]
print mode(train['material']).mode[0]
print mode(train['build_year']).mode[0]
print mode(train['num_room']).mode[0]
print mode(train['kitch_sq']).mode[0]
print mode(train['state']).mode[0]
print mode(train['industrial_km']).mode[0]


train['life_sq'].fillna(mode(train['life_sq']).mode[0], inplace=True)
train['floor'].fillna(mode(train['floor']).mode[0], inplace=True)
train['max_floor'].fillna(mode(train['max_floor']).mode[0], inplace=True)
train['material'].fillna(mode(train['material']).mode[0], inplace=True)
train['build_year'].fillna(mode(train['build_year']).mode[0], inplace=True)
train['num_room'].fillna(mode(train['num_room']).mode[0], inplace=True)
train['kitch_sq'].fillna(mode(train['kitch_sq']).mode[0], inplace=True)
train['state'].fillna(mode(train['state']).mode[0], inplace=True)
train['industrial_km'].fillna(mode(train['industrial_km']).mode[0], inplace=True)

19
3.0
17.0
1.0
2014.0
2.0
1.0
2.0
0.394107795


In [155]:
train.apply(lambda x: type(x[0]))

price_doc                           <type 'int'>
full_sq                             <type 'int'>
life_sq                           <type 'float'>
floor                             <type 'float'>
max_floor                         <type 'float'>
material                          <type 'float'>
build_year                        <type 'float'>
num_room                          <type 'float'>
kitch_sq                          <type 'float'>
state                             <type 'float'>
product_type                        <type 'int'>
sub_area                            <type 'int'>
indust_part                       <type 'float'>
school_education_centers_raion      <type 'int'>
sport_objects_raion                 <type 'int'>
culture_objects_top_25_raion        <type 'int'>
oil_chemistry_raion                 <type 'int'>
metro_min_avto                    <type 'float'>
green_zone_km                     <type 'float'>
industrial_km                     <type 'float'>
kremlin_km          

In [145]:
# Impute missing values in test set
print mode(test['life_sq']).mode[0]
print mode(test['build_year']).mode[0]
print mode(test['state']).mode[0]
print mode(test['income_per_cap']).mode[0]
print mode(test['salary']).mode[0]
print mode(test['labor_force']).mode[0]
print mode(test['unemployment']).mode[0]
print mode(test['employment']).mode[0]


test['life_sq'].fillna(mode(test['life_sq']).mode[0], inplace=True)
test['build_year'].fillna(mode(test['build_year']).mode[0], inplace=True)
test['state'].fillna(mode(test['state']).mode[0], inplace=True)
test['income_per_cap'].fillna(mode(test['income_per_cap']).mode[0], inplace=True)
test['salary'].fillna(mode(test['salary']).mode[0], inplace=True)
test['labor_force'].fillna(mode(test['labor_force']).mode[0], inplace=True)
test['unemployment'].fillna(mode(test['unemployment']).mode[0], inplace=True)
test['employment'].fillna(mode(test['employment']).mode[0], inplace=True)

1.0
2017.0
2.0
102709.0
64310.0
7067.470079
0.017708287
0.733598301


# House Feature Engineering

In [ ]:
#House features
train['kitch_full'] = train.kitch_sq/train.full_sq
train['kitch_life'] = train.kitch_sq/train.life_sq
train['rootfullsq']= np.sqrt(train.full_sq)
train['life_full'] = train.life_sq/train.full_sq
train['extra_full'] = (train.full_sq-train.life_sq)/train.full_sq
train['extra_life'] = (train.full_sq-train.life_sq)/train.life_sq
train['extra_on_life'] = train.full_sq/train.life_sq
train['rel_floor'] = train.floor/train.max_floor
train['build_age'] = train.timestamp.dt.year - train.build_year
train['unitprice'] = train.price_doc/train.full_sq

#Raion features
train['pop_density']=train.raion_popul/train.area_m
train['traffic'] = train.metro_km_avto/train.metro_min_avto # higher speed, lower the traffic 
train['safety'] = train.female_f/train.full_all #higher percentage of women, better the safety
train['young'] = train.young_all/train.full_all #percentage of young people
train['old'] = train.elder_all/train.full_all # percentage of old people
#train.under_14 = train.0_13_all/ train.full_all # percentage of under 14 yrs


In [146]:
# Mini macro
mini_macro = macro.loc[:,['timestamp',
'oil_urals',
'gdp_quart',
'cpi',
'usdrub',
'micex_cbi_tr',
'labor_force']]

In [147]:
# Create lag timestamps for one month, three months, 6 months, and 12 months in train
# Join with appropriate macro

mini_macro.columns = ['one_month_lag','oil_urals_1', 'gdp_quart_1', 'cpi_1', 
                      'usdrub_1', 'micex_cbi_tr_1', 'labor_force_1']
train['one_month_lag'] = train['timestamp'] - pd.to_timedelta('30 days')
train = pd.merge(train, mini_macro, how = 'left', on = 'one_month_lag')

# 3 month lage
mini_macro.columns = ['three_month_lag','oil_urals_3', 'gdp_quart_3', 'cpi_3', 
                      'usdrub_3', 'micex_cbi_tr_3', 'labor_force_3']
train['three_month_lag'] = train['timestamp'] - pd.to_timedelta('90 days')
train = pd.merge(train, mini_macro, how = 'left', on = 'three_month_lag')

# 6 month lage
mini_macro.columns = ['six_month_lag','oil_urals_6', 'gdp_quart_6', 'cpi_6', 
                      'usdrub_6', 'micex_cbi_tr_6', 'labor_force_6']
train['six_month_lag'] = train['timestamp'] - pd.to_timedelta('180 days')
train = pd.merge(train, mini_macro, how = 'left', on = 'six_month_lag')

# 12 month lage
mini_macro.columns = ['twelve_month_lag','oil_urals_12', 'gdp_quart_12', 'cpi_12', 
                      'usdrub_12', 'micex_cbi_tr_12', 'labor_force_12']
train['twelve_month_lag'] = train['timestamp'] - pd.to_timedelta('360 days')
train = pd.merge(train, mini_macro, how = 'left', on = 'twelve_month_lag')


In [148]:
# Create lag timestamps for one month, three months, 6 months, and 12 months in test
# Join with appropriate macro

mini_macro.columns = ['one_month_lag','oil_urals_1', 'gdp_quart_1', 'cpi_1', 
                      'usdrub_1', 'micex_cbi_tr_1', 'labor_force_1']
test['one_month_lag'] = test['timestamp'] - pd.to_timedelta('30 days')
test = pd.merge(test, mini_macro, how = 'left', on = 'one_month_lag')

# 3 month lage
mini_macro.columns = ['three_month_lag','oil_urals_3', 'gdp_quart_3', 'cpi_3', 
                      'usdrub_3', 'micex_cbi_tr_3', 'labor_force_3']
test['three_month_lag'] = test['timestamp'] - pd.to_timedelta('90 days')
test = pd.merge(test, mini_macro, how = 'left', on = 'three_month_lag')

# 6 month lage
mini_macro.columns = ['six_month_lag','oil_urals_6', 'gdp_quart_6', 'cpi_6', 
                      'usdrub_6', 'micex_cbi_tr_6', 'labor_force_6']
test['six_month_lag'] = test['timestamp'] - pd.to_timedelta('180 days')
test = pd.merge(test, mini_macro, how = 'left', on = 'six_month_lag')

# 12 month lage
mini_macro.columns = ['twelve_month_lag','oil_urals_12', 'gdp_quart_12', 'cpi_12', 
                      'usdrub_12', 'micex_cbi_tr_12', 'labor_force_12']
test['twelve_month_lag'] = test['timestamp'] - pd.to_timedelta('360 days')
test = pd.merge(test, mini_macro, how = 'left', on = 'twelve_month_lag')


In [149]:
# Find delta for oil
train['delta_oil_1'] = (train['oil_urals'] - train['oil_urals_1']) / train['oil_urals']
train['delta_oil_3'] = (train['oil_urals'] - train['oil_urals_3']) / train['oil_urals']
train['delta_oil_6'] = (train['oil_urals'] - train['oil_urals_6']) / train['oil_urals']
train['delta_oil_12'] = (train['oil_urals'] - train['oil_urals_12']) / train['oil_urals']

In [150]:
# find delta for usdrub exchange rate
train['delta_usdrub_1'] = (train['usdrub'] - train['usdrub_1']) / train['usdrub']
train['delta_usdrub_3'] = (train['usdrub'] - train['usdrub_3']) / train['usdrub']
train['delta_usdrub_6'] = (train['usdrub'] - train['usdrub_6']) / train['usdrub']
train['delta_usdrub_12'] = (train['usdrub'] - train['usdrub_12']) / train['usdrub']

In [151]:
# find delta of labor force
train['delta_labor_force_1'] = (train['labor_force'] - train['labor_force_1']) / train['labor_force']
train['delta_labor_force_3'] = (train['labor_force'] - train['labor_force_3']) / train['labor_force']
train['delta_labor_force_6'] = (train['labor_force'] - train['labor_force_6']) / train['labor_force']
train['delta_labor_force_12'] = (train['labor_force'] - train['labor_force_12']) / train['labor_force']

In [152]:
# now do the same for test
test['delta_oil_1'] = (test['oil_urals'] - test['oil_urals_1']) / test['oil_urals']
test['delta_oil_3'] = (test['oil_urals'] - test['oil_urals_3']) / test['oil_urals']
test['delta_oil_6'] = (test['oil_urals'] - test['oil_urals_6']) / test['oil_urals']
test['delta_oil_12'] = (test['oil_urals'] - test['oil_urals_12']) / test['oil_urals']

test['delta_usdrub_1'] = (test['usdrub'] - test['usdrub_1']) / test['usdrub']
test['delta_usdrub_3'] = (test['usdrub'] - test['usdrub_3']) / test['usdrub']
test['delta_usdrub_6'] = (test['usdrub'] - test['usdrub_6']) / test['usdrub']
test['delta_usdrub_12'] = (test['usdrub'] - test['usdrub_12']) / test['usdrub']

test['delta_labor_force_1'] = (test['labor_force'] - test['labor_force_1']) / test['labor_force']
test['delta_labor_force_3'] = (test['labor_force'] - test['labor_force_3']) / test['labor_force']
test['delta_labor_force_6'] = (test['labor_force'] - test['labor_force_6']) / test['labor_force']
test['delta_labor_force_12'] = (test['labor_force'] - test['labor_force_12']) / test['labor_force']

In [166]:
train.iloc[:,[29]]

,trc_sqm_5000
0,4036616
1,2034942
2,1572990
3,942180
4,3503058
5,2010320
6,1656529
7,2197401
8,1051314
9,1404650


In [154]:
train = add_date_features(train)
test = add_date_features(test)

In [ ]:
#np.sum(test.isnull())
# Replace test missings with 0
test = test.fillna(0)

# Another model with different predictors


In [156]:
from sklearn.ensemble import RandomForestRegressor

X2 = train.iloc[:,1:len(train)]
y2 = np.log(train[[0]])
x2_test = test

sample_leaf_options = [1,5,10,50,100,200,500]

for leaf_size in sample_leaf_options :
    rfr = RandomForestRegressor(n_estimators=1000, oob_score=True, n_jobs = -1,
                                   verbose = 1, random_state = 50, min_samples_leaf = leaf_size)
    # Train the model using the training sets and check score
    rfr.fit(X2, y2.values.ravel())
    print "Leaf Size: %d" % (leaf_size)
    print rfr.score(X2, y2)


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  9.9min finished


Leaf Size: 1


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    3.6s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    4.6s finished


0.919083355108


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  6.4min finished


...

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  1.9min finished


Leaf Size: 500


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.0s


0.360275692561


[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.2s finished


In [168]:
# Leaf size 50 seems ideal

rfr50 = RandomForestRegressor(n_estimators=1000, oob_score=True, n_jobs = -1,
                            verbose = 1, random_state = 50, min_samples_leaf = 50)

rfr50.fit(X2, y2.values.ravel())

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  4.1min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=50,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=True, random_state=50,
           verbose=1, warm_start=False)

In [169]:
importances = rfr50.feature_importances_
std = np.std([tree.feature_importances_ for tree in rfr50.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X2.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

Feature ranking:
1. feature 0 (0.514083)
2. feature 29 (0.112188)
3. feature 16 (0.056031)
4. feature 30 (0.028500)
5. feature 27 (0.026660)
6. feature 23 (0.021184)
7. feature 28 (0.015782)
8. feature 19 (0.013874)
9. feature 25 (0.011957)
10. feature 24 (0.010693)
11. feature 21 (0.009207)
12. feature 11 (0.009156)
13. feature 18 (0.008666)
14. feature 13 (0.008415)
15. feature 17 (0.007586)
16. feature 64 (0.007466)
17. feature 10 (0.007410)
18. feature 22 (0.007343)
19. feature 58 (0.007192)
20. feature 5 (0.006790)
21. feature 20 (0.006340)
22. feature 52 (0.005848)
23. feature 51 (0.005495)
24. feature 70 (0.005386)
25. feature 63 (0.004595)
26. feature 6 (0.004498)
27. feature 57 (0.004142)
28. feature 40 (0.004048)
29. feature 2 (0.004035)
30. feature 36 (0.004000)
31. feature 12 (0.003853)
32. feature 69 (0.003122)
33. feature 7 (0.003112)
34. feature 9 (0.002925)
35. feature 26 (0.002800)
36. feature 35 (0.002739)
37. feature 34 (0.002277)
38. feature 79 (0.002264)
39. featur

In [181]:
predicted = rfr50.predict(x2_test)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.2s finished


In [182]:
type(predicted)

numpy.ndarray

In [183]:
predicted

array([ 15.5067522 ,  15.89177719,  15.47833869, ...,  15.33877152,
        15.4944138 ,  15.89824875])

In [184]:
len(predicted)

7662

In [185]:
submission = pd.read_csv("/Users/andrewrubino/Desktop/test.csv", parse_dates = ['timestamp'])

In [618]:
submission.head()
print len(submission)

7662


In [186]:
submission['index1'] = submission.index
submission.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [187]:
predicted_price = pd.DataFrame(predicted)

In [188]:
predicted_price['index1'] = predicted_price.index

,0,index1
0,15.506752,0
1,15.891777,1
2,15.478339,2
3,15.612939,3
4,15.456021,4


In [189]:
predicted_price.columns = ['price_doc', 'index1']

,price_doc,index1
0,15.506752,0
1,15.891777,1
2,15.478339,2
3,15.612939,3
4,15.456021,4


In [190]:
submission = pd.merge(submission, predicted_price, how = 'left', on = 'index1')

In [192]:
submission = submission.loc[:,['id','price_doc']]

In [193]:
submission['price_doc'] = np.exp(submission['price_doc'])

In [194]:
submission.set_index('id', inplace=True)

In [195]:
submission.to_csv("rfr3_submission.csv")

# One more model
               

In [197]:
X3 = train.loc[:,['month','year','day','year_month','num_room','kremlin_km','cafe_count_5000',
               'metro_min_avto','additional_education_km','build_year',
               'industrial_km','state','delta_oil_1', 'delta_oil_3', 'delta_oil_6',
               'delta_oil_12', 'delta_labor_force_1','delta_labor_force_3',
               'delta_labor_force_6', 'delta_labor_force_12', 'delta_usdrub_1',
               'delta_usdrub_3','delta_usdrub_6', 'delta_usdrub_12', 'gdp_quart']]
y3 = np.log(train[[0]])
x3_test = test.loc[:,['month','year','day','year_month','num_room','kremlin_km','cafe_count_5000',
               'metro_min_avto','additional_education_km','build_year',
               'industrial_km','state','delta_oil_1', 'delta_oil_3', 'delta_oil_6',
               'delta_oil_12', 'delta_labor_force_1','delta_labor_force_3',
               'delta_labor_force_6', 'delta_labor_force_12', 'delta_usdrub_1',
               'delta_usdrub_3','delta_usdrub_6', 'delta_usdrub_12', 'gdp_quart']]

sample_leaf_options = [1,5,10,50,100,200,500]

for leaf_size in sample_leaf_options :
    rfr3 = RandomForestRegressor(n_estimators=1000, oob_score=True, n_jobs = -1,
                                   verbose = 1, random_state = 69, min_samples_leaf = leaf_size)
    # Train the model using the training sets and check score
    rfr3.fit(X3, y3.values.ravel())
    print "Leaf Size: %d" % (leaf_size)
    print rfr3.score(X3, y3)


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.9min finished


Leaf Size: 1


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    2.8s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    3.5s finished


0.903289750202


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   58.0s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.0min finished


Leaf Size: 5


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    2.3s finished


0.683725911737


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   45.8s


KeyboardInterrupt: 

In [ ]:
# Make submission a function
id_test = test['id']
y_pred = np.round(predicted * 0.99)
df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})